In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (80.9 kB/s)
Reading package li

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-25 23:20:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.8’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-05-25 23:20:39 (10.2 MB/s) - ‘postgresql-42.2.9.jar.8’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in the Amazon Electronics review from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option("header", "true").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [ ]:
# Drop duplicates and incomplete rows
print(f"Row count from file: {df.count()}")
df = df.dropna()
print(f"Row count after Nan are removed: {df.count()}")
df = df.dropDuplicates()
print(f"Row count after Nan and duplicates are removed: {df.count()}")

Row count from file: 3093869
Row count after Nan are removed: 3093660
Row count after Nan and duplicates are removed: 3093660


In [ ]:
# Examine the schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Import struct fields that will be used
from pyspark.sql.types import (
    StringType, BooleanType, IntegerType, FloatType, DateType
)

In [ ]:
# Change the data type of the "star_rating" and "review_date" columns
df2 = df.select(df.marketplace,
                df.customer_id,
                df.review_id,
                df.product_id,
                df.product_title,
                df.product_parent,
                df.product_category,
                (df.star_rating.cast(IntegerType())).alias("star_rating"),
                df.helpful_votes,
                df.total_votes,
                df.vine,
                df.verified_purchase,
                df.review_headline,
                df.review_body,
                (df.review_date.cast(DateType())).alias("review_date"),
)

In [ ]:
# Examine the schema
df2.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Create review_id_table dataframe
review_id_table_df = df2.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
# Drop duplicates
review_id_table_df = review_id_table_df.dropDuplicates()
# Preview dataframe
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000K9CAN0EKG|   43145645|B000CBURC8|     276230742| 2009-12-27|
|R100F99FWQQZQ6|   50606935|B000MVJ1M0|     925089317| 2013-08-29|
|R102CUJU7VMK3C|   18984735|B004T0MW7Q|     588302616| 2014-11-03|
|R1033Q9UZXJ4WR|   16237314|B001RB24R8|      57778913| 2011-05-29|
|R103E3TSP5OHRG|   52510829|B00HEZK8DS|      58002863| 2015-04-16|
|R1044WJ4P84PHT|   51021728|B004OBZ2Z4|     303975127| 2013-01-25|
|R10495CVUTYM3Q|   49125070|B000ONZTMW|     714943063| 2009-07-18|
|R104A1KXWYC2FP|   30490008|B003C2T03G|     666314203| 2012-11-21|
|R104P6XW3MAR0R|   47003464|B004477OB8|     244937560| 2012-02-07|
|R104PKZPK4QLMM|   11199595|B005LMJUDU|      95610284| 2013-01-30|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Examine the schema
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Create products dataframe
products_df = df2.select(["product_id", "product_title"])
# Drop duplicates
products_df = products_df.dropDuplicates()
# Preview dataframe
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0097BEE9Q|Apple iPod nano 1...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B00KXDS4VQ|enKo products HDM...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# Examine the schema
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Create vine_table dataframe
vine_table_df = df2.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
# Preview dataframe
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100J4AOWOWCTX|          1|            4|          4|   N|
|R100N7Y79EQNCL|          3|            0|          0|   N|
|R1015SVKAKV7EQ|          5|            0|          0|   N|
|R101BIRNEZ17XW|          4|            0|          2|   N|
|R102PG7ZKS5IFS|          1|            4|         12|   N|
|R10398PM75STU0|          5|            2|          2|   N|
|R103QXL0PLKBVO|          5|            0|          0|   N|
|R103Z5DGS3VBRK|          5|            0|          0|   N|
|R104271C2Q2DCM|          5|            0|          0|   N|
|R1049GRPORPPOG|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# Examine the schema
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# Create customers dataframe
temp = df2.select(["customer_id"])
customers_df = temp.groupBy("customer_id").count().alias("customer_count")
# Rename "count" column to "customer_count"
customers_df = customers_df.withColumnRenamed("count","customer_count")
# Change the data type of the "customer_count" column
customers_df2 = customers_df.select(customers_df.customer_id,
                                    (customers_df.customer_count.cast(IntegerType())).alias("customer_count"),
                                    )
# Drop duplicates
customers_df2 = customers_df2.dropDuplicates()
# Preview dataframe
customers_df2.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   41626553|             1|
|   46134234|             4|
|   13705152|             1|
|   44413267|             1|
|   36644422|             2|
|    8061365|             1|
|   49196304|             9|
|   11587470|             1|
|   25538476|             3|
|   46889381|             3|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Examine the schema
customers_df2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/amazon_reviews_electronics"
config = {"user":"<insert username>",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to RDS
review_id_table_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write DataFrame to RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write DataFrame to RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)

In [ ]:
# Write DataFrame to RDS
customers_df2.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)